In [14]:
import numpy as np
import re
import os
import json
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from model_loader.config import embedding_loader, generation_loader
from chroma_db import ChromaVectorDB

class HybridSearcher:
    def __init__(self, embedding_model, chunk_size=200, chunk_overlap=50, 
                 persist_directory="./chroma_db", collection_name="Gray"):
        """
        하이브리드 검색 클래스 초기화
        
        Args:
            embedding_model: 임베딩 모델
            chunk_size (int): 청크 크기
            chunk_overlap (int): 청크 간 중복 크기
            persist_directory (str): 벡터 DB 저장 경로
            collection_name (str): 벡터 DB 컬렉션 이름
        """
        self.embedding_model = embedding_model
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.chunks = None
        self.chunk_metadata = None
        self.bm25_index = None
        self.vector_index = None
        
        # ChromaDB 벡터 DB 클래스 초기화 - 동일한 임베딩 모델 사용
        self.vector_db = ChromaVectorDB(
            embedding_model=self.embedding_model,
            persist_directory=persist_directory,
            collection_name=collection_name
        )
        
        # 검색 결과 비교용 플래그
        self.debug_mode = False
        
        # 프롬프트 로드
        self.complexity_prompt = self._load_prompt("prompts/en/complex/complex_prompt.txt")
        self.decompose_prompt = self._load_prompt("prompts/en/decompose/decompose_prompt.txt")

    def _load_prompt(self, file_path):
        """프롬프트 파일 로드"""
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                return f.read()
        except Exception as e:
            print(f"프롬프트 파일 로드 중 오류 발생: {e}")
            # 기본 프롬프트 반환
            if "complexity" in file_path:
                return "질문을 분석하여 복합적인 질문인지 판단하세요. 복합적인 질문은 여러 하위 질문으로 분해할 수 있습니다. '예' 또는 '아니오'로만 답변하세요."
            elif "decompose" in file_path:
                return "다음 복합적인 질문을 여러 개의 간단한 하위 질문으로 분해하세요. JSON 형식으로 하위 질문 목록을 반환하세요."

    def load_document(self, file_path):
        """문서 로드 및 청크 분할"""
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        
        # 문서를 청크로 분할
        self.chunks, self.chunk_metadata = self._split_into_chunks_with_metadata(content)
        
        # BM25 인덱스 생성
        tokenized_chunks = [self._simple_tokenize(chunk) for chunk in self.chunks]
        self.bm25_index = BM25Okapi(tokenized_chunks)
        
        # 벡터 임베딩 생성 및 인메모리 인덱스 구축
        self.vector_index = self.embedding_model.encode(self.chunks)
        
        # ChromaDB에 문서 추가
        # 문서 ID 생성
        doc_name = os.path.basename(file_path)
        doc_ids = [f"{doc_name}_{i}" for i in range(len(self.chunks))]
        
        # 메타데이터에 페이지 정보 포함
        metadatas = [{"page": meta["page"], "source": doc_name, "chunk_index": i} 
                    for i, meta in enumerate(self.chunk_metadata)]
        
        # 벡터 DB에 문서 추가
        self.vector_db.add_documents(
            documents=self.chunks,
            metadatas=metadatas,
            ids=doc_ids
        )
        
        return len(self.chunks)
    
    def search(self, query, top_n=5, alpha=0.5, use_vector_db=True):
        """
        하이브리드 검색 수행
        
        Args:
            query (str): 검색 쿼리
            top_n (int): 반환할 결과 수
            alpha (float): BM25와 벡터 검색 결과 결합 비율 (0에 가까울수록 벡터 검색 중시)
            use_vector_db (bool): ChromaDB 벡터 DB 사용 여부
        
        Returns:
            list: 검색 결과 리스트
        """
        if self.chunks is None or self.bm25_index is None:
            raise ValueError("문서가 로드되지 않았습니다. load_document()를 먼저 호출하세요.")
        
        if not isinstance(query, str) :
            query = str(query)
            
        # BM25 검색 수행
        bm25_scores = self.bm25_index.get_scores(self._simple_tokenize(query))
        
        # 두 검색 방식 모두 수행 (디버그 모드)
        query_embedding = self.embedding_model.encode([query])[0]
        memory_scores = cosine_similarity([query_embedding], self.vector_index)[0]
        
        if use_vector_db:
            # ChromaDB를 사용한 벡터 검색
            db_results = self.vector_db.search(query, top_n=len(self.chunks))
            
            # 결과를 벡터 점수로 변환
            vector_scores = np.zeros(len(self.chunks))
            for res in db_results:
                # 문서 ID에서 인덱스 추출
                chunk_id = res["id"]
                if "_" in chunk_id:
                    try:
                        chunk_index = int(chunk_id.split("_")[-1])
                        if chunk_index < len(self.chunks):
                            # 거리를 유사도로 변환 (코사인 거리는 1 - 코사인 유사도)
                            if res["distance"] is not None:
                                # ChromaDB가 코사인 거리를 사용하는 경우
                                similarity = 1.0 - res["distance"]
                                vector_scores[chunk_index] = similarity
                    except ValueError:
                        continue
            
            # 디버그 모드에서 점수 비교
            if self.debug_mode:
                print("\n벡터 검색 점수 비교:")
                for i in range(min(5, len(self.chunks))):
                    print(f"Chunk {i}: Memory={memory_scores[i]:.4f}, ChromaDB={vector_scores[i]:.4f}, 차이={memory_scores[i]-vector_scores[i]:.4f}")
        else:
            # 메모리 내 벡터 검색 수행 (기존 방식)
            vector_scores = memory_scores
        
        # 검색 결과 결합
        combined_scores = self._combine_scores(bm25_scores, vector_scores, alpha)
        
        # 상위 N개 결과 반환
        top_indices = np.argsort(combined_scores)[-top_n:][::-1]
        results = [
            {
                "chunk": self.chunks[i],
                "score": combined_scores[i],
                "bm25_score": bm25_scores[i],
                "vector_score": vector_scores[i],
                "memory_score": memory_scores[i],  # 디버깅용
                "index": i,
                "page": self.chunk_metadata[i]["page"]
            }
            for i in top_indices
        ]
        
        return results

    def _split_into_chunks_with_metadata(self, text):
        """텍스트를 청크로 분할하고 페이지 정보를 메타데이터로 유지하는 함수"""
        chunks = []
        chunk_metadata = []
        
        # 페이지 패턴 정규식 (####으로 시작하는 페이지 헤더)
        page_pattern = re.compile(r'####\s*Page (\d+)')
        
        # 텍스트를 줄 단위로 처리
        lines = text.split('\n')
        current_page = "unknown"
        current_chunk = ""
        
        for line in lines:
            # 페이지 헤더 확인
            page_match = page_pattern.match(line)
            
            if page_match:
                # 새 페이지 시작
                # 현재 청크가 있으면 저장
                if current_chunk.strip():
                    chunks.append(current_chunk.strip())
                    chunk_metadata.append({"page": current_page})
                    current_chunk = ""
                
                # 새 페이지 번호 설정
                current_page = page_match.group(1)
                continue
            
            # 현재 청크에 라인 추가
            current_chunk += line + "\n"
            
            # 청크 크기 확인
            if len(current_chunk) >= self.chunk_size:
                chunks.append(current_chunk.strip())
                chunk_metadata.append({"page": current_page})
                current_chunk = ""  # 새 청크 시작
        
        # 마지막 청크 처리
        if current_chunk.strip():
            chunks.append(current_chunk.strip())
            chunk_metadata.append({"page": current_page})
        
        # 너무 작은 청크 결합 (메타데이터 유지)
        i = 0
        while i < len(chunks) - 1:
            if len(chunks[i]) + len(chunks[i+1]) < self.chunk_size:
                # 같은 페이지인 경우에만 결합
                if chunk_metadata[i]["page"] == chunk_metadata[i+1]["page"]:
                    chunks[i] = chunks[i] + "\n\n" + chunks[i+1]
                    chunks.pop(i+1)
                    chunk_metadata.pop(i+1)
                else:
                    i += 1
            else:
                i += 1
        
        return chunks, chunk_metadata
    
    def _simple_tokenize(self, text):
        """텍스트를 간단히 토크나이징하는 함수"""
        if not isinstance(text, str):
            text = str(text)
        return re.findall(r'\w+', text.lower())
    
    def _combine_scores(self, bm25_scores, vector_scores, alpha=0.5):
        """BM25와 벡터 검색 점수를 결합"""
        # 점수 정규화
        if np.max(bm25_scores) > 0:
            bm25_scores = bm25_scores / np.max(bm25_scores)
        if np.max(vector_scores) > 0:
            vector_scores = vector_scores / np.max(vector_scores)
        
        # 가중 평균 계산
        combined = alpha * bm25_scores + (1 - alpha) * vector_scores
        return combined
    
    def get_chunks_with_page_info(self, indices=None):
        """청크와 페이지 정보 반환"""
        if indices is None:
            return [(chunk, self.chunk_metadata[i]["page"]) for i, chunk in enumerate(self.chunks)]
        else:
            return [(self.chunks[i], self.chunk_metadata[i]["page"]) for i in indices if i < len(self.chunks)]
    
    # 1. 질문이 복합적인지 확인하는 메서드
    def is_complex_question(self, question):
        """
        LLM을 사용하여 질문이 복합적인지 판단하는 메서드
        
        Args:
            question (str): 사용자 질문
            
        Returns:
            bool: 복합적인 질문이면 True, 아니면 False
        """
        prompt = self.complexity_prompt.format(question=question)
        response = generation_loader.generate(prompt)
        
        # '예' 또는 'Yes'가 응답에 포함되어 있으면 복합적인 질문으로 간주
        response = response.lower().strip()
        print(f"###############질문이 복잡한가요? : {response}")
        return '예' in response or 'yes' in response
    
    # 2. 복합적인 질문을 하위 질문으로 분해하는 메서드
    def decompose_question(self, complex_question):
        """
        LLM을 사용하여 복합적인 질문을 여러 개의 하위 질문으로 분해하는 메서드
        
        Args:
            complex_question (str): 복합적인 사용자 질문
            
        Returns:
            list: 하위 질문 목록
        """
        prompt = self.decompose_prompt.format(question=complex_question)
        response = generation_loader.generate(prompt)

        try:
            # JSON 형식으로 반환된 하위 질문 파싱
            # JSON 블록 추출 (```json과 ```로 감싸져 있을 수 있음)
            json_match = re.search(r'```json\s*(.+?)\s*```', response, re.DOTALL)
            if json_match:
                response = json_match.group(1)
            
            # 중괄호 블록 추출
            json_match = re.search(r'(\{.+?\})', response, re.DOTALL)
            if json_match:
                response = json_match.group(1)
                
            sub_questions_data = json.loads(response)
            # 다양한 JSON 형식 처리
            if isinstance(sub_questions_data, list):
                return sub_questions_data
            elif isinstance(sub_questions_data, dict):
                if "questions" in sub_questions_data:
                    return sub_questions_data["questions"]
                elif "subQuestions" in sub_questions_data:
                    return sub_questions_data["subQuestions"]
                else:
                    # 딕셔너리의 값들을 리스트로 반환
                    return list(sub_questions_data.values())
                
        except (json.JSONDecodeError, AttributeError) as e:
            print(f"하위 질문 파싱 오류: {e}")
            print(f"LLM 응답: {response}")
            
            # 파싱 실패 시 줄바꿈을 기준으로 질문 추출 시도
            questions = []
            for line in response.split('\n'):
                line = line.strip()
                if line and ('?' in line or '질문' in line):
                    # 숫자, 점, 괄호 등의 접두어 제거
                    cleaned_line = re.sub(r'^[\d\.\)\-\s]+', '', line).strip()
                    if cleaned_line:
                        questions.append(cleaned_line)
            
            if questions:
                return questions
            # 단일 질문으로 처리
            return [complex_question]

In [27]:
from model_loader.config import *
from save_utils import *

class CarManualQA:
    def __init__(self, generation_loader, data_folder="./data/split_file", 
                 prompt_path_ko="./prompts/ko/generation/gemma3/generation_prompt2.txt", 
                 prompt_path_en="./prompts/en/generation/gemma3/generation_prompt2.txt", 
                 result_path="./result/5월12일/gemma3",
                 use_vector_db=True, 
                 persist_directory="./chroma_db",
                 collection_name="Gray",
                 language="en"):
        """
        자동차 매뉴얼 Q&A 시스템 초기화
        
        Args:
            generation_loader: 텍스트 생성 모델 로더
            data_folder (str): 분할된 데이터 파일들이 있는 폴더 경로
            prompt_path (str): 프롬프트 템플릿 파일 경로
            result_path (str): 결과를 저장할 경로
            use_vector_db (bool): ChromaDB 벡터 DB 사용 여부
            persist_directory (str): 벡터 DB 저장 경로
            collection_name (str): 벡터 DB 컬렉션 이름
        """
        self.data_folder = data_folder
        self.result_path = result_path
        self.use_vector_db = use_vector_db
        self.language = language

        if collection_name == "Gray" :
            collection_name = f"Gray_{language}"
        
        # 하이브리드 검색기 초기화 - ChromaDB 지원 버전
        self.searcher = HybridSearcher(
            embedding_model=embedding_loader,
            chunk_size=200, 
            chunk_overlap=50,
            persist_directory=persist_directory,
            collection_name=collection_name
        )
        
        self.loader = generation_loader
        self.prompt_path_ko = prompt_path_ko
        self.prompt_path_en = prompt_path_en

        if language=="ko" :
            self.prompt_template = self._load_prompt(prompt_path_ko)
        else :
            self.prompt_template = self._load_prompt(prompt_path_en)
        
        # 카테고리별 파일 매핑
        if language == "en" :
            self.category_to_file = {
                "16": "GrayAnatomy_Formatted.md"
            }
        else :
            self.category_to_file = {
                "16": "full.txt"
            }
        
        try :
            from translate import translate_en_to_ko
            self.translate_en_to_ko = translate_en_to_ko
        except ImportError :
            print("translate.py 모듈을 임포트할 수 없습니다.")
            self.translate_en_to_ko = translate_en_to_ko

        # 이미 처리된 파일 추적
        self.loaded_files = set()
        
        # 전체 파일을 벡터 DB에 미리 로드할지 여부
        self.preload_all = False
        
        # 디버그 모드
        self.debug_mode = False
        
    def preload_documents(self):
        """
        모든 카테고리의 파일을 미리 벡터 DB에 로드
        """
        print("모든 문서를 벡터 DB에 로드 중...")
        for category, filename in self.category_to_file.items():
            file_path = os.path.join(self.data_folder, filename)
            if os.path.exists(file_path):
                print(f"카테고리 {category}: {filename} 로드 중...")
                self._load_document(file_path)
                self.loaded_files.add(file_path)
            else:
                print(f"[경고] 파일을 찾을 수 없습니다: {file_path}")
        
        print(f"총 {len(self.loaded_files)}개 파일이 벡터 DB에 로드되었습니다.")
        self.preload_all = True

    def _load_document(self, file_path):
        """
        문서를 로드하여 검색기에 추가
        
        Args:
            file_path (str): 로드할 파일 경로
            
        Returns:
            int: 로드된 청크 수
        """
        # 이미 로드된 파일이면 건너뛰기
        if file_path in self.loaded_files and self.preload_all:
            if self.debug_mode:
                print(f"이미 로드된 파일입니다: {file_path}")
            return 0
        
        # 파일 로드 및 벡터 DB에 추가
        num_chunks = self.searcher.load_document(file_path)
        
        # 파일 추적 목록에 추가
        self.loaded_files.add(file_path)
        
        if self.debug_mode:
            print(f"파일 로드 완료: {file_path} ({num_chunks}개 청크)")
        
        return num_chunks

    def filter_relevant_content(self, query, search_results, threshold=0.7, top_k=5):
        """
        LLM을 사용하여 검색 결과에서 질문과 관련이 있는 내용을 평가하고 
        관련성 점수가 높은 상위 k개 결과만 반환
        
        Args:
            query (str): 사용자 질의
            search_results (list): 검색 결과 목록
            threshold (float): 관련성 점수 임계값 (0.0~1.0)
            top_k (int): 반환할 상위 결과 개수
            
        Returns:
            list: 점수가 높은 상위 k개의 필터링된 관련 정보 목록
        """
        scored_results = []
        
        if self.debug_mode:
            print(f"검색 결과 {len(search_results)}개에 대한 관련성 필터링 시작")
        
        # 각 검색 결과에 대해 관련성 평가
        for result in search_results:
            # 프롬프트 구성 - 관련성 평가용
            relevance_prompt = f"""
    다음은 사용자의 질문입니다:
    "{query}"
    다음은 검색된 텍스트 정보입니다:
    "{result['chunk']}"
    위 텍스트가 사용자 질문에 얼마나 관련이 있는지 평가해주세요.
    평가는 다음과 같이 응답해주세요:
    1. 관련성 점수: 0.0 ~ 1.0 사이의 숫자 (1.0이 가장 관련성 높음)
    2. 이유: 관련성이 높거나 낮은 이유를 간략하게 설명
    응답 형식:
    {{
    "score": 0.0~1.0,
    "reason": "평가 이유"
    }}
    """
            
            relevance_result = self.loader.generate(relevance_prompt)
            
            try:
                # JSON 응답 파싱 (정확한 JSON이 아닐 수 있으므로 예외 처리)
                import re
                import json
                
                # JSON 형식 추출 시도
                json_match = re.search(r'\{.*"score".*:.*,.*"reason".*:.*\}', relevance_result, re.DOTALL)
                if json_match:
                    relevance_data = json.loads(json_match.group(0))
                    score = float(relevance_data.get("score", 0))
                else:
                    # 숫자만 추출 시도
                    score_match = re.search(r'score"?\s*:?\s*(\d+\.\d+|\d+)', relevance_result)
                    score = float(score_match.group(1)) if score_match else 0.0
                
                # 점수와 함께 결과 저장 (임계값 이상인 것만)
                if score >= threshold:
                    if self.debug_mode:
                        print(f"관련성 높음 (점수: {score:.2f}): {result['page']}페이지")
                    # 원본 결과에 점수 정보 추가
                    result_with_score = result.copy()
                    result_with_score['relevance_score'] = score
                    scored_results.append(result_with_score)
                else:
                    if self.debug_mode:
                        print(f"관련성 낮음 (점수: {score:.2f}): {result['page']}페이지")
                    
            except Exception as e:
                # 파싱 실패 시 안전을 위해 낮은 점수로 포함
                if self.debug_mode:
                    print(f"관련성 평가 파싱 오류: {str(e)}")
                result_with_score = result.copy()
                result_with_score['relevance_score'] = 0.1  # 낮은 기본 점수
                scored_results.append(result_with_score)
        
        # 점수를 기준으로 내림차순 정렬
        sorted_results = sorted(scored_results, key=lambda x: x['relevance_score'], reverse=True)
        
        # 상위 k개 결과만 선택
        top_results = sorted_results[:top_k]
        
        if self.debug_mode:
            print(f"필터링 결과: 총 {len(search_results)}개 중 {len(scored_results)}개가 임계값 통과")
            print(f"상위 {min(top_k, len(top_results))}개 결과만 선택됨")
            for i, res in enumerate(top_results):
                print(f"  {i+1}위: 점수 {res['relevance_score']:.2f} - {res['page']}페이지")
        
        # 결과가 없는 경우 원본의 일부라도 반환
        if not top_results and search_results:
            if self.debug_mode:
                print(f"필터링 결과가 없어 원본 상위 결과 {min(top_k, len(search_results))}개 포함")
            top_results = search_results[:min(top_k, len(search_results))]
        
        return top_results
    
    def _format_filtered_results(self, filtered_results):
        """
        필터링된 결과를 문자열로 변환
        
        Args:
            filtered_results (list): 필터링된 검색 결과 리스트
            
        Returns:
            str: 포맷된 문자열
        """
        result_str = ""
        
        for item in filtered_results:
            result_str += f"# 질문: {item['sub_question']}\n\n"
            
            # results가 이미 점수로 정렬되었다고 가정
            for res in item['results']:
                page_info = res['page']
                score_info = ""
                if 'relevance_score' in res:
                    score_info = f" (관련성: {res['relevance_score']:.2f})"
                
                result_str += f"## {page_info}페이지{score_info}\n"
                result_str += f"{res['chunk']}\n\n"
        
        return result_str

    def generate_response(self, query, category, top_n=5, alpha=0.5, target_language=None):
        """
        질의에 대한 응답 생성
        
        Args:
            query (str): 사용자 질의
            category (str): 검색할 카테고리
            top_n (int): 검색 결과 수
            alpha (float): BM25와 벡터 검색 가중치 (높을수록 BM25 중시)
            
        Returns:
            dict: 생성된 응답 및 메타데이터
        """
        try:
            response_language = target_language if target_language else self.language
            source_language = self.language

            # 파일 로드 확인
            filename = self.category_to_file[category]
            file_path = os.path.join(self.data_folder, filename)
            
            # 해당 파일이 존재하지 않는 경우
            if not os.path.exists(file_path):
                error_msg = f"[오류] 파일을 찾을 수 없습니다: {file_path}"
                print(error_msg)
                return {"답변": error_msg, "후처리": error_msg}
            
            # 파일 로드 (이미 로드된 경우 건너뜀)
            if not self.preload_all:
                self._load_document(file_path)
            
            # 단계 1: 질문 복잡성 판단
            is_complex = self.searcher.is_complex_question(query)
            
            if self.debug_mode:
                print(f"질문 복잡성 분석: {'복합적인 질문' if is_complex else '단순 질문'}")
            
            if is_complex:
                # 복합적인 질문 처리
                # 단계 2: 질문 분해
                sub_questions = self.searcher.decompose_question(query)
                print(f"하위 질문 분해 결과: {sub_questions}")
                
                # 모든 하위 질문에 대한 검색 결과를 저장할 리스트
                all_search_results = []
                all_filtered_results = []

                # 단계 3: 각 하위 질문에 대해 검색 수행
                for i, sub_q in enumerate(sub_questions):
                    if self.debug_mode:
                        print(f"하위 질문 {i+1} 검색 중: '{sub_q}'")
                    
                    if not isinstance(query, str) :
                        query = str(query)
                    # 하위 질문에 대한 검색 수행
                    search_results = self.searcher.search(
                        query=sub_q, 
                        top_n=top_n, 
                        alpha=alpha,
                        use_vector_db=self.use_vector_db
                    )
                    filtered_results = self.filter_relevant_content(query, search_results)

                    # 검색 결과 저장
                    all_search_results.append({
                        "sub_question": sub_q,
                        "results": search_results
                    })

                    all_filtered_results.append({
                        "sub_question" : sub_q,
                        "results" : filtered_results
                    })

                # 컨텍스트 구성 - 모든 하위 질문 결과 통합
                context = ""
                for i, sr in enumerate(all_filtered_results):
                    context += f"\n### 하위 질문 {i+1}: {sr['sub_question']}\n"
                    
                    for _, res in enumerate(sr["results"]):
                        context += f"#### Page {res['page']}\n"
                        context += f"{res['chunk']}\n\n"
                
                # 프롬프트 구성 - 복합 질문용
                prompt = self.prompt_template.format(context=context, query=query)
                
            else:
                # 단순 질문 처리 - 기존 방식대로 검색
                search_results = self.searcher.search(
                    query=query, 
                    top_n=top_n, 
                    alpha=alpha,
                    use_vector_db=self.use_vector_db
                )
                
                filtered_results = self.filter_relevant_content(query, search_results)

                # 컨텍스트 구성 - 단순 질문용
                context = "\n\n".join([f"#### Page {result['page']}\n{result['chunk']}" for result in filtered_results])
                
                # 프롬프트 구성 - 단순 질문용
                prompt = self.prompt_template.format(context=context, query=query)
            
            # 응답 생성 파트
            if hasattr(self.loader, "tokenizer"):
                # Huggingface 모델 사용
                tokenizer = self.loader.tokenizer
                model = self.loader.model
                
                # 인풋 토크나이즈
                input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
                attention_mask = (input_ids != tokenizer.pad_token_id).long().to(model.device)
                
                # 텍스트 생성
                output = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=400,
                    temperature=0.3,
                    do_sample=False,
                    top_p=0.85,
                    repetition_penalty=1.2,
                    early_stopping=True,
                    num_beams=3,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                )
                generated_ids = output[0][input_ids.shape[-1]:]
                raw_answer = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
            else:
                # OpenAI 등 API 기반 모델 사용 (Ollama 포함)
                raw_answer = self.loader.generate(prompt)
            
            # 응답 후처리
            final_answer = self._extract_answer_content(raw_answer)
            final_answer = self._remove_chinese_characters(final_answer)
            
            translated_answer = None
            if source_language == "en" and response_language == "ko" :
                if self.translate_en_to_ko :
                    try :
                        translated_answer = self.translate_en_to_ko(final_answer)
                        if self.debug_mode :
                            print("영어 -> 한국어 번역 완료")
                    except Exception as e :
                        print(f"번역 중 오류 발생 : {e}")
                        translated_answer = f"[번역 오류] {final_answer}"
                else :
                    translated_answer = f"[번역 모듈 없음] {final_answer}"
            # 결과 준비
            response = {
                "답변": raw_answer,
                "후처리": final_answer,
                "문서 일부": context,
                "question_en": query,
                "answer_en": raw_answer,
                "is_complex": is_complex,
                "필터링_결과": all_filtered_results
            }
            
            if translated_answer :
                response["번역된 답변"] = translated_answer
                
            # 복합 질문인 경우 하위 질문 정보 추가
            if is_complex:
                response["sub_questions"] = sub_questions
                response["sub_search_results"] = all_search_results
                response["filtered_search_results"] = all_filtered_results
            else:
                response["검색_결과"] = search_results
                response["필터링_결과"] = all_filtered_results
            
            # 결과 저장
            try:
                # result_path가 없으면 상위 스코프나 기본값으로 설정
                result_path = getattr(self, 'result_path', '../result')
                
                # 알파값으로 폴더 경로 생성
                # alpha_str = f"{alpha:.1f}"
                # db_type = "chromadb" if self.use_vector_db else "memory"
                # question_type = "complex" if is_complex else "simple"
                # result_path = os.path.join(result_path, f"{db_type}_{question_type}_alpha_{alpha_str}")
                
                # 폴더가 없으면 생성
                os.makedirs(result_path, exist_ok=True)
                
                filtered_results_str = ""
                filtered_results_str = self._format_filtered_results(response["필터링_결과"])

                # 파일 저장
                if self.debug_mode:
                    print(f"저장 시도: {result_path}")
                save_response_to_file(
                    query=query,
                    answer=response["답변"],
                    final_answer=response["후처리"],
                    context=response["문서 일부"],
                    folder=result_path,
                    filtered_results=filtered_results_str
                )
                if self.debug_mode:
                    print(f"저장 완료: {result_path}")
            except Exception as e:
                print(f"결과 저장 중 오류 발생: {e}")
            
            return response
            
        except Exception as e:
            error_msg = f"[생성 오류] {str(e)}"
            print(error_msg)
            import traceback
            traceback.print_exc()
            return {"답변": error_msg, "후처리": error_msg}
    
    def _load_prompt(self, path):
        """프롬프트 템플릿 로드"""
        try:
            with open(path, "r", encoding="utf-8") as f:
                return f.read()
        except Exception as e:
            print(f"[ERROR] 프롬프트 로드 실패: {e}")
            raise
    
    
    def _extract_answer_content(self, text):
        """응답에서 답변 부분만 추출"""
        pattern = r"(?:<\|?|<|)?\|?answer\|?(?:\|?>|>)?(.*?)(?:<\|?|<|)?\|?endanswer\|?(?:\|?>|>)?"
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        return match.group(1).strip() if match else text.strip()
    
    def _remove_chinese_characters(self, text):
        """중국어 문자 제거"""
        return re.sub(r'[\u4E00-\u9FFF]', '', text)
    
    def get_collection_stats(self):
        """벡터 DB 컬렉션 통계 조회"""
        if hasattr(self.searcher, 'vector_db') and hasattr(self.searcher.vector_db, 'get_collection_stats'):
            return self.searcher.vector_db.get_collection_stats()
        return {"error": "벡터 DB 통계를 조회할 수 없습니다."}
    
    def toggle_vector_db(self, use_vector_db=None):
        """벡터 DB 사용 여부 전환"""
        if use_vector_db is not None:
            self.use_vector_db = use_vector_db
        else:
            self.use_vector_db = not self.use_vector_db
        
        print(f"벡터 DB 사용 여부: {self.use_vector_db}")
        return self.use_vector_db
    
    def set_debug_mode(self, debug_mode=True):
        """디버그 모드 설정"""
        self.debug_mode = debug_mode
        return self.debug_mode

In [16]:
import os
import chromadb
from chromadb.utils import embedding_functions
import re
import pathlib

class ChromaVectorDB:
    """ChromaDB를 이용한 벡터 DB 관리 클래스"""
    
    def __init__(self, embedding_model, persist_directory="./chroma_db", collection_name="Gray"):
        """
        ChromaDB 초기화
        
        Args:
            embedding_model: 임베딩 모델 (SentenceTransformer 또는 호환 래퍼)
            persist_directory (str): 벡터 DB가 저장될 디렉토리 경로
            collection_name (str): 컬렉션 이름
        """
        # 디렉토리 생성 확인
        os.makedirs(persist_directory, exist_ok=True)
        
        # 임베딩 모델 저장
        self.embedding_model = embedding_model
        
        # ChromaDB 클라이언트 설정
        self.client = chromadb.PersistentClient(path=persist_directory)
        
        # 사용자 정의 임베딩 함수 생성 - 기존 embedding_model을 활용
        self.embedding_function = CustomEmbeddingFunction(self.embedding_model)
        
        # 컬렉션 생성 또는 가져오기
        try:
            self.collection = self.client.get_collection(
                name=collection_name,
                embedding_function=self.embedding_function
            )
            print(f"기존 컬렉션 '{collection_name}'을 로드했습니다.")
        except Exception as e:
            print(f"컬렉션 로드 중 오류 발생: {e}")
            print(f"새 컬렉션 '{collection_name}'을 생성합니다.")
            self.collection = self.client.create_collection(
                name=collection_name,
                embedding_function=self.embedding_function
            )
    
    def add_documents(self, documents, metadatas=None, ids=None):
        """
        문서를 벡터 DB에 추가
        
        Args:
            documents (list): 문서 텍스트 리스트
            metadatas (list, optional): 각 문서에 대한 메타데이터 리스트
            ids (list, optional): 각 문서에 대한 고유 ID 리스트
        
        Returns:
            int: 추가된 문서 수
        """
        if ids is None:
            # 고유한 ID 생성 (timestamp + index)
            import time
            timestamp = int(time.time())
            ids = [f"doc_{timestamp}_{i}" for i in range(len(documents))]
        
        # 메타데이터가 제공되지 않은 경우 빈 딕셔너리 생성
        if metadatas is None:
            metadatas = [{} for _ in range(len(documents))]
        
        # 문서 임베딩 미리 계산 - 디버깅 용도
        # embeddings = self.embedding_model.encode(documents)
        
        # 문서 추가 (임베딩은 임베딩 함수가 자동 계산)
        self.collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )
        
        return len(documents)
    
    ########################## 새로 추가한 함수 ##########################
    def load_document_file(self, file_path, chunk_size=None):
        """
        파일을 로드하여 벡터 DB에 저장
        
        Args:
            file_path (str): 로드할 파일 경로
            chunk_size (int, optional): 청크 분할 크기 (None이면 페이지 단위로 분할)
            
        Returns:
            int: 추가된 청크 수
        """
        # 파일 확장자 확인
        _, ext = os.path.splitext(file_path)
        
        # 파일 읽기
        try:
            print(f"파일 로드 중: {file_path}")
            text_content = pathlib.Path(file_path).read_text(encoding="utf-8")
            print(f"파일 크기: {len(text_content)} 문자")
        except Exception as e:
            print(f"파일 로드 오류: {e}")
            return 0
        
        # 청크 분할 방식 결정
        chunks = []
        metadatas = []
        
        if chunk_size:
            # 지정된 청크 크기로 분할
            current_pos = 0
            while current_pos < len(text_content):
                chunk = text_content[current_pos:current_pos + chunk_size]
                chunks.append(chunk)
                metadatas.append({"source": file_path, "chunk_index": len(chunks)})
                current_pos += chunk_size
            
            print(f"크기 기반 분할: {len(chunks)}개 청크 생성")
        else:
            # 페이지 단위 분할 (#### Page X 패턴 사용)
            page_pattern = r'(####\s+Page\s+\d+\s*\n(?:[\s\S]*?)(?=####\s+Page\s+\d+\s*\n|$))'
            page_chunks = re.findall(page_pattern, text_content)
            
            if page_chunks:
                # 페이지 패턴 찾음
                for page_chunk in page_chunks:
                    # 페이지 번호 추출
                    page_match = re.match(r'####\s+Page\s+(\d+)', page_chunk)
                    page_num = page_match.group(1) if page_match else "unknown"
                    
                    chunks.append(page_chunk)
                    metadatas.append({"source": file_path, "page": f"Page {page_num}"})
                
                print(f"페이지 기반 분할: {len(chunks)}개 페이지 찾음")
            else:
                # 페이지 패턴 못찾음 - 문단 단위로 분할
                paragraphs = re.split(r'\n\s*\n', text_content)
                paragraphs = [p.strip() for p in paragraphs if p.strip()]
                
                for i, para in enumerate(paragraphs):
                    chunks.append(para)
                    metadatas.append({"source": file_path, "paragraph": i+1})
                
                print(f"문단 기반 분할: {len(chunks)}개 문단 생성")
        
        # 청크가 없으면 전체 텍스트를 하나의 청크로 처리
        if not chunks:
            chunks = [text_content]
            metadatas = [{"source": file_path, "full_document": True}]
            print("분할 실패: 전체 텍스트를 하나의 청크로 처리")
        
        # 벡터 DB에 추가
        added_count = self.add_documents(
            documents=chunks,
            metadatas=metadatas
        )
        
        print(f"벡터 DB에 {added_count}개 청크 추가 완료")
        return added_count
    ##################################################################
    
    def search(self, query, top_n=5, metadata_filter=None):
        """
        벡터 유사도 기반 검색 수행
        
        Args:
            query (str): 검색 쿼리
            top_n (int): 반환할 결과 수
            metadata_filter (dict, optional): 메타데이터 필터링 조건
        
        Returns:
            dict: 검색 결과
        """
        # 진단용 - 쿼리 임베딩 계산
        query_embedding = self.embedding_model.encode([query])[0]
        print(f"쿼리 임베딩 shape: {query_embedding.shape}")
        
        # ChromaDB를 통한 검색
        results = self.collection.query(
            query_texts=[query],
            n_results=top_n,
            where=metadata_filter,
            include=["documents", "metadatas", "distances", "embeddings"]
        )
        
        # 결과 포맷팅
        formatted_results = []
        for i in range(len(results["documents"][0])):
            formatted_results.append({
                "chunk": results["documents"][0][i],
                "id": results["ids"][0][i],
                "metadata": results["metadatas"][0][i],
                "distance": results["distances"][0][i] if "distances" in results else None,
                # "embedding": results["embeddings"][0][i] if "embeddings" in results else None
            })
        
        return formatted_results
    
    def get_all_document_embeddings(self, ids=None):
        """
        저장된 모든 문서의 임베딩 조회 (디버깅 용도)
        """
        result = self.collection.get(ids=ids, include=["embeddings", "documents"])
        return result
    
    def get_document_by_id(self, doc_id):
        """ID로 문서 조회"""
        result = self.collection.get(ids=[doc_id])
        if result["documents"]:
            return {
                "chunk": result["documents"][0],
                "metadata": result["metadatas"][0]
            }
        return None
    
    def delete_document(self, doc_id):
        """ID로 문서 삭제"""
        self.collection.delete(ids=[doc_id])
    
    def delete_collection(self):
        """컬렉션 삭제"""
        self.client.delete_collection(self.collection.name)
    
    def get_collection_stats(self):
        """컬렉션 통계 조회"""
        count = self.collection.count()
        return {
            "document_count": count,
            "collection_name": self.collection.name
        }
    
class CustomEmbeddingFunction(embedding_functions.EmbeddingFunction):
    """기존 임베딩 모델을 ChromaDB에서 사용하기 위한 래퍼 클래스"""
    
    def __init__(self, embedding_model):
        """
        생성자
        
        Args:
            embedding_model: 임베딩 모델 (encode 메서드 제공)
        """
        self.embedding_model = embedding_model
    
    def __call__(self, texts):
        """
        텍스트를 임베딩 벡터로 변환
        
        Args:
            texts: 인코딩할 텍스트 리스트
        
        Returns:
            임베딩 벡터 리스트
        """
        return self.embedding_model.encode(texts).tolist()

In [17]:
# from model_loader.config import generation_loader
# import numpy as np
# import time

# question_list = [
#                 "애가 문 못열게 하는거 어케함?", 
#                 "운전석에서 문 다 잠그는거 할 수 있나?",
#                 "차키 배터리 뭐사야돼?",
#                 "하이패스에 빨간불 들어오는데 왜이럼?",
#                 "시트 너무 낮은데 어떻게 조절함?",
#                 "창문 올라가다 마는데 이거 왜이래",
#                 "도난방지인가 그거 어떻게 끄냐?",
#                 "카드 꽂았는데 하이패스 안되는데 어케함",
#                 "열선 버튼 누르면 언제 꺼지냐?",
#                 "스마트키 물에 빠졌는데 써도돼?",
#                 "하이패스 후불로 하고싶은데 어디다가 말해야대?",
#                 "차키 안에 두고 내려서 다른 키로 열었는데 안에 있던 키 작동이 안돼.",
#                 "차키 배터리 어따가 버려야되냐? 일반쓰레기인가?",
#                 "차 달리다가 자동으로 잠기는거 몇킬로로 달릴때 잠기냐?",
#                 "지금 차 친구한테 팔려고 하는데 안에 하이패스는 어케 해야되냐",
#                 "하이패스에서 카드를 확인하십시오 라고 나오는데 뭐가문제야",
#                 "하이패스 등록하려는데 서류 뭐내야돼?",
#                 "뒷자리 애가 창문열고 장난치는데 창문 잠그는방법 없어?",
#                 "임산부는 벨트 매야되나? 위험하지않나?",
#                 "차 산지 5년 다되가는데 에어백도 점검해야되나?"
#                 ]
# category_list = ["16"] * 20

# generation_loader = generation_loader
# # result_base_path = "../result"
# qa_system = CarManualQA(
#         generation_loader=generation_loader,
#         data_folder="./data/split_file",
#         prompt_path="./prompts/ko/generation/gemma3/generation_prompt.txt",
#         result_path="./result/5월13일/gemma3",
#         use_vector_db=True,
#         persist_directory="./chroma_db",
#         collection_name="carmanual_documents"
#     )


    
# for q, c in zip(question_list, category_list):
#     print(f"\n새 쿼리: {q}, 카테고리: {c}")
    
    
#     # 검색 결과 가져오기 (alpha 값 전달)
#     # search_results = test_search_only(q, c, alpha=alpha_formatted)
#     # context = "\n\n".join([result["chunk"] for result in search_results])
    
#     start_time = time.time()
#     # 응답 생성 (alpha 값 전달)
#     chroma_response = qa_system.generate_response(q, c, top_n=5, alpha=0.3)
#     end_time = time.time()
    
    
    
#     elapsed_time = end_time - start_time

#     print(f"추론시간 : {elapsed_time}")
#     print(f"LLM 답변: {chroma_response['후처리']}")

# 5/14

# PyMuPDF4LLM

In [ ]:
import pymupdf4llm
import pathlib

pdf_path="./data/GrayAnatomyPDF.pdf"
md_text=pymupdf4llm.to_markdown(pdf_path)
pathlib.Path("GrayAnatomy.md").write_bytes(md_text.encode())

In [ ]:
# import pymupdf4llm
# import pathlib
# import fitz

# pdf_path="../data/GrayAnatomyPDF.pdf"
# doc = fitz.open(pdf_path)
# total_pages = len(doc)

# all_text = ""
# for page_num in range(total_pages) :
#     page_header = f"#### Page {page_num + 1}\n\n"
#     md_text = pymupdf4llm.to_markdown(pdf_path, pages=[page_num])

#     all_text += page_header + md_text + "\n\n"

#     print(f"처리중 : {page_num + 1}/{total_pages} 페이지")

# pathlib.Path("GrayAnatomyPage.md").write_bytes(all_text.encode())

# print(md_text)

처리중 : 1/852 페이지


KeyboardInterrupt: 

In [1]:
import pymupdf4llm
import pathlib
import json

class PyMuPDFEncoder(json.JSONEncoder):
    def default(self, obj):
        # Rect 객체를 튜플로 변환
        if hasattr(obj, '__str__') and 'Rect' in str(type(obj)):
            return tuple(obj)  # Rect 객체를 튜플로 변환
        # IRect 객체 처리
        elif hasattr(obj, '__str__') and 'IRect' in str(type(obj)):
            return tuple(obj)
        # Matrix 객체 처리
        elif hasattr(obj, '__str__') and 'Matrix' in str(type(obj)):
            return tuple(obj)
        # 기타 PyMuPDF 객체를 문자열로 변환
        elif hasattr(obj, '__str__') and ('pymupdf' in str(type(obj)).lower() or 'fitz' in str(type(obj)).lower()):
            return str(obj)
        # 기본 JSON 인코더 사용
        return super().default(obj)

if __name__ == "__main__" :
    pdf_path = "../data/dongseo_report.pdf"
    page_data = pymupdf4llm.to_markdown(pdf_path, page_chunks=True)
    print(f"총 {len(page_data)}개의 페이지 정보가 추출되었습니다.")


    # JSON 파일로 전체 페이지 정보 저장
    pathlib.Path("dongseo.json").write_text(
        json.dumps(page_data, indent=2, cls=PyMuPDFEncoder), 
        encoding="utf-8"
    )
    print(f"page : {page_data}")
    # 모든 페이지의 텍스트만 결합하여 마크다운 파일로 저장
    formatted_pages = []
    for i, page in enumerate(page_data) :
        print(page["text"])
        formatted_pages.append(f"### page {i+1}\n{page['text']}")

    combined_text = "\n\n--\n\n".join(formatted_pages)
    pathlib.Path("dongseo_report.md").write_text(combined_text, encoding="utf-8")

총 61개의 페이지 정보가 추출되었습니다.
page : [{'metadata': {'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': '', 'encryption': None, 'file_path': '../data/dongseo_report.pdf', 'page_count': 61, 'page': 1}, 'toc_items': [], 'tables': [], 'images': [], 'graphics': [], 'text': '## **Heat Exchanger Status**\n\n\n2025-06-02\n\n\n**Client:**\n\n**Site:**\n\n**Exchanger:**\n**Section:**\n\n\nEWP\n\nDonghae #2\nCondenser\n\nA - Inlet\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n15 PLG: Plugged\n0 OBS: Obstructed\n\n374 DNT: Dented\n\n0 WLL: Wall loss long\n\n\n0 80 < Defect <= 100\n\n0 60 < Defect <= 80\n\n1 40 < Defect <= 60\n\n0 20 < Defect <= 40\n\n\n1 0 < Defect <= 20\n\n5835 No Defect Detected\n\n0 Not Tested\n\n', 'words': []}, {'metadata': {'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': '

In [ ]:
import fitz  # PyMuPDF 라이브러리는 'fitz'로 import 합니다.
import pathlib

# PDF 파일들이 있는 디렉토리 경로
pdf_directory = pathlib.Path("../data/SGMP_PDF/")

# 처리할 PDF 파일 목록
pdf_files = [
    pdf_directory / "SGMP_appendix.pdf",
    pdf_directory / "SGMP.pdf"
]

for pdf_path in pdf_files:
    if not pdf_path.exists():
        print(f"오류: 파일이 존재하지 않습니다 - {pdf_path}")
        continue

    try:
        # PDF 문서 열기
        doc = fitz.open(pdf_path)
        
        # 모든 페이지에서 텍스트 추출
        full_text = ""
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            full_text += page.get_text("text") # "text" 인자는 일반 텍스트 추출을 의미합니다.
            
            # 각 페이지 사이에 줄바꿈 추가 (선택 사항)
            if page_num < doc.page_count - 1:
                full_text += "\n\n" 
        
        # 추출된 텍스트를 TXT 파일로 저장
        # 원본 PDF 파일 이름에서 확장자를 .txt로 변경
        txt_path = pdf_path.with_suffix(".txt")
        
        txt_path.write_text(full_text, encoding="utf-8")
        
        print(f"'{pdf_path.name}' 파일에서 텍스트를 추출하여 '{txt_path.name}'로 저장했습니다.")
        
        # 문서 닫기
        doc.close()

    except Exception as e:
        print(f"'{pdf_path.name}' 파일 처리 중 오류 발생: {e}")

print("모든 PDF 파일 처리 완료.")

'SGMP_appendix.pdf' 파일에서 텍스트를 추출하여 'SGMP_appendix.txt'로 저장했습니다.
'SGMP.pdf' 파일에서 텍스트를 추출하여 'SGMP.txt'로 저장했습니다.
모든 PDF 파일 처리 완료.


: 

In [21]:
# print(page_data)

In [22]:
# import pprint

# print("메타데이터 키:")
# pprint.pprint(page_data[0]["metadata"].keys())

In [23]:
# for i, page in enumerate(page_data):
#     print(f"=== 페이지 {i+1} 정보 ===")
    
#     # 1. 메타데이터 출력
#     metadata = page["metadata"]
#     print(f"메타데이터: {metadata}")
#     print(f"페이지 번호: {metadata['page']}")
#     print(f"전체 페이지 수: {metadata['page_count']}")
    
#     # 2. 목차 항목 출력
#     if page["toc_items"]:
#         print("\n목차 항목:")
#         for item in page["toc_items"]:
#             level, title, page_num = item
#             print(f"  레벨 {level}: {title} (페이지 {page_num})")
    
    
#     # 4. 이미지 정보 출력
#     if page["images"]:
#         print("\n이미지 정보:")
#         for j, img in enumerate(page["images"]):
#             print(f"  이미지 {j+1}: {img}")
    
#     # 5. 그래픽 정보 출력
#     if page["graphics"]:
#         print("\n그래픽 정보:")
#         for j, graphic in enumerate(page["graphics"]):
#             print(f"  그래픽 {j+1}: {graphic}")
    
#     # 6. 텍스트 일부 출력 (처음 100자)
#     text_preview = page["text"][:100] + "..." if len(page["text"]) > 100 else page["text"]
#     print(f"\n텍스트 미리보기: {text_preview}")
    
#     # 7. 단어 정보 출력 (extract_words=True 설정했을 경우)
#     if "words" in page and page["words"]:
#         print(f"\n단어 수: {len(page['words'])}")
#         print(f"첫 5개 단어: {page['words'][:5]}")
    
#     print("\n" + "-"*50 + "\n")

In [24]:
# formatted_pages = []

# for i, page in enumerate(page_data) :
#     try :
#         page_number = page["metadata"]["page"]
#     except KeyError :
#         page_number = i + 1

#     page_header = f"#### Page {page_number}\n\n"

#     page_text = page["text"]

#     formatted_pages.append(page_header + page_text)

#     print(f"페이지 {page_number} 작성")

# full_text = "\n\n".join(formatted_pages)

# output_file = "GrayAnatomy_Formatted.md"
# pathlib.Path(output_file).write_text(full_text, encoding="utf-8")
# print("파일 저장 완료")

In [25]:
vector_db = ChromaVectorDB(
    embedding_model=embedding_loader,
    persist_directory="./chroma_db",
    collection_name="Gray"
)

# 파일 하나만 로드하면 자동으로 처리해서 벡터 DB에 저장
vector_db.load_document_file("./data/GrayAnatomy_Formatted.md")

기존 컬렉션 'Gray'을 로드했습니다.
파일 로드 중: ./data/GrayAnatomy_Formatted.md
파일 크기: 3899416 문자
페이지 기반 분할: 852개 페이지 찾음
벡터 DB에 852개 청크 추가 완료


852

In [28]:
from model_loader.config import *
from chroma_db import *
import chromadb
import numpy as np
import time

questions = [
    "How do the processes of ossification differ between the skull and the vertebral column, and what are the developmental implications of these differences?",
    "Compare and contrast the ovum and spermatozoon in terms of their development, maturation process, and contribution to fertilization, explaining how their specialized features facilitate their respective roles.",
    "How does the structure and development of the vertebral column demonstrate both functional adaptation and evolutionary history, and what role does the notochord play in these processes?",
    "How do the various stages of embryonic implantation, placental development, and fetal membrane formation collectively establish the maternal-fetal interface, and what are the critical structural and functional relationships that develop during this process?",
    "What are the primary and secondary curves of the vertebral column, how do they develop chronologically, and what functional advantages do they provide for bipedal locomotion?",
    "How does the skull's composition of dermal and cartilaginous bones reflect both its developmental origins and its functional requirements, and what are the key differences in the ossification patterns of these two bone types?",
    "Explain the process of fertilization and early embryonic development, describing how the male and female pronuclei interact to form the segmentation nucleus, and how this leads to the formation of the blastocyst and subsequent implantation.",
    "How does the development of the thoracic cage from its embryonic origins to its adult form demonstrate both functional adaptation and regional specialization, and what are the key differences between true and false ribs in terms of their structure and functions?",
    "Compare and contrast the development and structural features of the cervical, thoracic, and lumbar vertebrae, explaining how their specialized characteristics relate to their differing functional roles.",
    "How does the structure of the placenta facilitate its multiple physiological functions while maintaining separation between maternal and fetal circulations, and what developmental stages are critical to establishing this complex organ?",
    "How do the primitive embryonic germ layers (ectoderm, mesoderm, and endoderm) contribute to the formation of the skeletal, nervous, and digestive systems, and what key developmental interactions occur between these systems?",
    "Compare the processes of intramembranous and intracartilaginous ossification, detailing their anatomical sites, cellular mechanisms, and developmental timelines, and explain how these different processes create the various bones of the skeletal system.",
    "Describe the embryonic development of the branchial region and explain how its derivatives contribute to adult craniofacial structures, detailing the specific fates of each branchial arch and the developmental abnormalities that can occur when this process is disrupted.",
    "What is the sequence of events that occurs during the maturation of an ovum, and how does this process differ from spermatogenesis in terms of cellular divisions, chromosome reduction, and the number and viability of resulting cells?",
    "Describe the development of the limbs from their initial appearance as limb buds to their final form, explaining the molecular signals, tissue interactions, and morphogenetic movements involved in this process, and how developmental abnormalities can lead to congenital limb defects.",
    "Compare the structure and ossification patterns of the vertebral column, sternum, and ribs, explaining how these components develop together to form the thoracic cage, and how variations in their development can lead to congenital abnormalities.",
    "How does the skull's development from its embryonic origins demonstrate both segmentation patterns similar to the vertebral column and unique craniofacial adaptations, and what are the key differences in ossification between the cranial base, vault, and facial bones?",
    "Describe the embryonic development of the body cavities, explaining how the coelom divides to form the pericardial, pleural, and peritoneal cavities, and how the diaphragm develops to separate the thoracic and abdominal compartments.",
    "How do the neural tube and notochord interact during development to establish the central nervous system and axial skeleton, and what are the key signaling molecules and inductive interactions that guide this process?",
    "Explain the processes of implantation and placentation, detailing the roles of the trophoblast, decidua, and extraembryonic membranes in establishing the maternal-fetal interface, and how abnormalities in these processes can lead to pregnancy complications.",
    "Compare and contrast the three parts of the axillary artery in terms of their anatomical relations and clinical significance, explaining how these relationships might impact surgical approaches and potential complications during procedures in the axilla.",
    "Describe the course and major branches of the brachial artery, explaining how its anatomical variations affect the collateral circulation after ligation of different segments of the artery, and discuss the clinical implications of these variations for traumatic injuries and surgical procedures in the upper limb.",
    "Analyze the arterial supply of the hand, detailing the formation and clinical significance of the superficial and deep palmar arches, their anastomotic patterns, and how these dual systems provide redundancy that affects treatment strategies for hand injuries and vascular disorders.",
    "Examine the pathway and variations of the radial artery from its origin to the formation of the deep palmar arch, discussing the surgical implications of these variations for procedures such as radial artery harvesting for coronary bypass grafting and the creation of arteriovenous fistulas for hemodialysis access.",
    "Compare and contrast the anatomical course, relations, and branching patterns of the ulnar and radial arteries in the forearm and hand, analyzing how these differences impact the clinical assessment and management of vascular injuries to these vessels.",
    "Analyze the anatomical basis of collateral circulation around the elbow joint following ligation or obstruction of the brachial artery, explaining how the network of anastomosing vessels ensures tissue viability and discussing the clinical implications of this vascular arrangement for traumatic injuries and surgical interventions.",
    "Describe the anatomical boundaries and contents of the axilla, explaining how the neurovascular structures are arranged within this space and discussing the clinical implications of these relationships for procedures such as axillary lymph node dissection, brachial plexus blocks, and approaches to the shoulder joint.",
    "Explain the origin, course, and branching pattern of the abdominal aorta, describing how its major visceral and parietal branches supply different organ systems, and analyzing the clinical significance of important vascular landmarks and anastomoses for surgical procedures and management of vascular diseases.",
    "Compare the anatomical relationships and branching patterns of the axillary and brachial arteries, discussing how these relationships provide landmarks for surgical approaches, and analyzing the implications of anatomical variations for diagnostic procedures, treatment of traumatic injuries, and vascular access in these regions.",
    "Describe the arterial supply to the rectum and anal canal, explaining how the superior, middle, and inferior rectal arteries form a complex anastomotic network, and analyze the clinical significance of this vascular arrangement for colorectal surgical procedures, management of hemorrhoids, and potential consequences of vascular compromise in this region.",
    "Analyze the vascular supply of the hand through the radial and ulnar arteries, explaining how their terminal branches form the superficial and deep palmar arches, and discuss the clinical implications of these anastomotic patterns for hand injuries, surgical approaches, and the assessment of hand perfusion.",
    "Describe the origin, course, and major branches of the internal iliac (hypogastric) artery, explaining how its anatomy differs in the fetus compared to the adult, and discuss the clinical significance of these vessels in pelvic surgery, trauma management, and vascular embolization procedures.",
    "Explain the branching pattern and distribution of the celiac trunk, detailing how its three major divisions (left gastric, splenic, and hepatic arteries) supply the upper abdominal organs, and analyze the clinical significance of their anatomical variations, collateral pathways, and relationships to surrounding structures.",
    "Compare and contrast the anatomical relationships and distribution patterns of the superior and inferior mesenteric arteries, discussing their embryological significance, anastomotic connections, and the clinical implications of these vascular arrangements for intestinal ischemia, surgical resections, and collateral circulation in occlusive disease.",
    "Analyze the anatomical relationships and variations of the renal arteries, explaining their importance in surgical approaches to the kidney, renal transplantation, and interventional procedures, while also discussing the clinical significance of accessory renal vessels and their potential role in renovascular hypertension.",
    "Describe the origin, course, and branches of the external iliac artery, explaining its relationships to surrounding structures and analyzing the clinical implications of these anatomical features for vascular access procedures, traumatic injuries, and surgical approaches to the lower abdomen and pelvis.",
    "Compare and contrast the arterial supply of the male and female pelvic organs, describing how the internal iliac (hypogastric) artery and its branches distribute blood to these structures, and discuss the clinical significance of these vascular patterns for surgical procedures, hemorrhage control, and interventional radiological techniques.",
    "Describe the anatomical course and branching pattern of the internal pudendal artery in males and females, explaining how this vessel supplies the perineum and external genitalia, and discuss the clinical significance of its relationships for urogenital and anorectal procedures, traumatic injuries, and erectile dysfunction.",
    "Explain the anatomical relationships and branching pattern of the thoracic aorta, describing how its branches supply the thoracic walls and viscera, and discuss the clinical significance of these vascular patterns in thoracic surgery, trauma management, and the development of collateral circulation in cases of aortic coarctation.",
    "Analyze the surgical approaches to the axillary artery and its branches, discussing the potential complications of these procedures, strategies for minimizing vascular injury, and the management of axillary artery trauma in relation to the surrounding neurovascular structures.",
    "How does the female urethra compare to the male urethra in terms of structure, length, and coats? What specific differences in epithelial lining exist between them?",
    "What is the developmental journey of the testes from fetal life to adulthood, and what abnormalities can occur during this process? How do these abnormalities affect fertility and health?",
    "What are the key components of the spermatic cord, and how do they relate to the blood supply, lymphatic drainage, and innervation of the testes? What clinical implications might arise from this anatomical arrangement?",
    "Describe the structure of the tunica vaginalis and its relationship to the peritoneum. How does it contribute to various testicular pathologies, especially different types of hydrocele, and what distinguishes these conditions from each other?",
    "Compare and contrast the structure and function of the vesiculae seminales, prostate gland, and bulbourethral glands. How do their secretions contribute to the composition of semen, and what are the clinical implications of diseases affecting these accessory glands?",
    "How is the penis structured for its dual role in urination and reproduction? What vascular and nervous mechanisms enable erection, and how does this relate to the anatomical organization of the corpora cavernosa and corpus spongiosum?",
    "What is the anatomical relationship between the ovaries, uterine tubes, and uterus? How does this arrangement facilitate conception, and what pathologies might develop when abnormalities occur in this system?",
    "How do the structural and functional changes of the uterus vary across a woman's lifecycle, from fetal development through puberty, pregnancy, and menopause? What cyclical changes occur during the menstrual cycle?",
    "Compare the anatomy of the vagina and its relationship to surrounding structures with that of the male urethra. How do these differences affect susceptibility to infections and other pathological conditions?",
    "What is the embryological origin and development of the ovaries? How does this relate to the structures found in the adult ovary, and what implications does this have for the development of ovarian pathologies?",
    "Describe the structure and function of the clitoris and penis as homologous organs. How do they compare in terms of erectile tissue arrangement, vascular supply, innervation, and embryological development? What implications does this have for sexual response and dysfunction?",
    "How do the vestibular bulbs in females compare anatomically and functionally to the corpus spongiosum/bulb of the penis in males? What is their role in sexual arousal, and how do they relate to the greater vestibular glands and other external genital structures?",
    "What is the anatomical and functional relationship between the mammary glands and the reproductive system? How do they develop embryologically, and what hormonal changes affect their structure and function throughout a woman's life from puberty through pregnancy, lactation, and menopause?",
    "How does the structure of the uterine tube facilitate the transport of ova and fertilization? What are the different segments of the tube, their specific functions, and how does ectopic pregnancy develop in relation to tubal anatomy?",
    "What is the structure and function of the prostate gland, and how does its anatomy relate to common pathologies? How do the different anatomical zones of the prostate correspond to the development of conditions like benign prostatic hyperplasia and prostate cancer?",
    "Compare and contrast the structure, development, and functions of the thyroid gland with the other major endocrine glands. How does the embryological origin of the thyroid relate to its adult position and function, and what common pathologies affect this gland?",
    "How do the anatomical features of the female reproductive system contribute to both normal fertility and common infertility issues? Consider the ovaries, fallopian tubes, uterus, cervix, and vagina in your analysis, and explain how structural or functional abnormalities in each can impact conception and pregnancy.",
    "How does the structure of the scrotum and its layers provide optimal temperature regulation for the testes? What vascular adaptations contribute to this function, and how do pathological conditions like varicocele affect testicular function?",
    "How do the anatomical and histological structures of the breast change throughout a woman's life, particularly during pregnancy and lactation? What cellular mechanisms enable milk production and ejection, and how does the transition from colostrum to mature milk occur?",
    "Compare and contrast the structure, embryological origin, and function of the male and female external genitalia. How do homologous structures differ in their adult form, and what hormonal factors influence their development and later function?"
]
category_list = ["16"] * 20

vector_db = ChromaVectorDB(
    embedding_model=embedding_loader,
    persist_directory="./chroma_db",
    collection_name="Gray"
)
vector_db.delete_collection()
print("기존 컬렉션 삭제 완료")


generation_loader = generation_loader
# result_base_path = "../result"
qa_system = CarManualQA(
        generation_loader=generation_loader,
        data_folder="./data",
        prompt_path_en="./prompts/en/generation/gemma3/generation_prompt.txt",
        result_path="./result/5월15일/en-gemma3",
        use_vector_db=True,
        persist_directory="./chroma_db",
        collection_name="Gray",
        language="en"
    )


    
for q, c in zip(questions, category_list):
    print(f"\n새 쿼리: {q}, 카테고리: {c}")
    
    
    # 검색 결과 가져오기 (alpha 값 전달)
    # search_results = test_search_only(q, c, alpha=alpha_formatted)
    # context = "\n\n".join([result["chunk"] for result in search_results])
    
    start_time = time.time()
    # 응답 생성 (alpha 값 전달)
    chroma_response = qa_system.generate_response(q, c, top_n=5, alpha=0.3, target_language="en")
    end_time = time.time()
    
    
    
    elapsed_time = end_time - start_time

    print(f"추론시간 : {elapsed_time}")
    print(f"LLM 답변: {chroma_response['후처리']}")

컬렉션 로드 중 오류 발생: Collection Gray does not exist.
새 컬렉션 'Gray'을 생성합니다.
기존 컬렉션 삭제 완료
기존 컬렉션 'Gray_en'을 로드했습니다.

새 쿼리: How do the processes of ossification differ between the skull and the vertebral column, and what are the developmental implications of these differences?, 카테고리: 16


Exception occurred invoking consumer for subscription e9571f84b0b3480399a67e9fda8c7afato topic persistent://default/default/e8d7264b-d15c-42cd-9678-ea5bdaa09df4 


: 

In [1]:
import time
from translate import *

# 번역 성능 평가를 위한 어려운 영어 문장들
complex_english_sentences = [
    "The ubiquitous nature of quantum entanglement challenges our fundamental understanding of non-locality and contextuality in physics.",
    "Despite the epistemological barriers inherent in extrapolating paleoclimatic data, researchers have ascertained robust correlations between carbon dioxide fluctuations and global temperature anomalies.",
    "The hermeneutic approach to understanding consciousness requires a phenomenological bracketing of ontological presuppositions about the mind-body problem.",
    "The intricate interplay between epigenetic modifications and transcriptional regulation underscores the dynamic nature of cellular differentiation during embryonic development.",
    "Contemporary architectural paradigms are increasingly embracing biomimetic design principles, synthesizing ecological sustainability with algorithmic optimization.",
    "The sociolinguistic implications of code-switching among multilingual communities reflect both pragmatic competence and cultural hybridity in post-colonial discourse.",
    "The implementation of meta-heuristic algorithms in combinatorial optimization problems often exhibits a performance-complexity tradeoff that necessitates domain-specific adaptations.",
    "Autoethnographic methodologies have challenged traditional notions of objectivity in anthropological research, foregrounding the reflexive positioning of the researcher within cultural narratives.",
    "The recursive nature of synthetic derivatives in financial markets exacerbated systemic risk during the 2008 crisis, exposing regulatory inadequacies in monitoring leverage ratios.",
    "The emergence of post-humanist philosophy interrogates the anthropocentric assumptions underlying conventional ethical frameworks, particularly regarding interspecies justice and technological integration."
]

def evaluate_translations():
    """번역 성능 평가 함수"""
    print("=== 영어 → 한국어 → 영어 번역 평가 ===\n")
    
    for i, sentence in enumerate(complex_english_sentences):
        print(f"[원문 {i+1}] {sentence}")
        
        # 영어 → 한국어 번역
        start_time = time.time()
        korean_translation = translate_en_to_ko(sentence)
        ko_time = time.time() - start_time
        
        print(f"[한국어 번역] {korean_translation}")
        print(f"번역 시간: {ko_time:.2f}초")
        
        # 한국어 → 영어 역번역
        start_time = time.time()
        back_translation = translate_ko_to_en(korean_translation)
        en_time = time.time() - start_time
        
        print(f"[영어 역번역] {back_translation}")
        print(f"번역 시간: {en_time:.2f}초")
        
        # 간단한 BLEU 스코어 계산 (단어 일치율)
        # original_words = set(sentence.lower().replace(".", "").replace(",", "").split())
        # back_words = set(back_translation.lower().replace(".", "").replace(",", "").split())
        # common_words = original_words.intersection(back_words)
        # word_match_rate = len(common_words) / len(original_words) if original_words else 0
        
        # print(f"단어 일치율: {word_match_rate:.2%}")
        # print("-" * 100 + "\n")

if __name__ == "__main__":
    print("NLLB 모델 로딩 중...")
    global model, tokenizer
    model, tokenizer = None, None
    model, tokenizer = get_translation_model()
    print("모델 로딩 완료!")
    
    evaluate_translations()

NLLB 모델 로딩 중...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

모델 로딩 완료!
=== 영어 → 한국어 → 영어 번역 평가 ===

[원문 1] The ubiquitous nature of quantum entanglement challenges our fundamental understanding of non-locality and contextuality in physics.


AttributeError: NllbTokenizerFast has no attribute get_lang_id

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import time

def load_madlad_model(model_name='./model/translate/madlad'):
    """
    MADLAD 모델과 토크나이저 로드
    
    Args:
        model_name (str): 사용할 MADLAD 모델 이름
    
    Returns:
        tuple: (model, tokenizer) - 로드된 모델과 토크나이저
    """
    model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto")
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    return model, tokenizer

def translate_text(text, source_lang, target_lang, model=None, tokenizer=None):
    """
    MADLAD 모델을 사용하여 텍스트 번역
    
    Args:
        text (str): 번역할 텍스트
        source_lang (str): 원본 언어 코드 (예: 'ko', 'en')
        target_lang (str): 대상 언어 코드 (예: 'en', 'ko')
        model: 사전 로드된 모델 (없으면 새로 로드)
        tokenizer: 사전 로드된 토크나이저 (없으면 새로 로드)
    
    Returns:
        str: 번역된 텍스트
    """
    # 모델과 토크나이저가 제공되지 않은 경우 로드
    if model is None or tokenizer is None:
        model, tokenizer = load_madlad_model()
    
    # 텍스트 앞에 대상 언어 토큰 추가
    prefix = f"<2{target_lang}> "
    input_text = prefix + text
    
    # 입력 텍스트를 토큰화
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)
    
    # 번역 생성
    outputs = model.generate(
        input_ids=input_ids,
        max_length=512,
        num_beams=4,
        length_penalty=0.6
    )
    
    # 출력 토큰을 텍스트로 디코딩
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return translated_text

def ko_to_en(text, model=None, tokenizer=None):
    """
    한국어를 영어로 번역
    
    Args:
        text (str): 번역할 한국어 텍스트
        model: 사전 로드된 모델 (없으면 새로 로드)
        tokenizer: 사전 로드된 토크나이저 (없으면 새로 로드)
    
    Returns:
        str: 번역된 영어 텍스트
    """
    return translate_text(text, 'ko', 'en', model, tokenizer)

def en_to_ko(text, model=None, tokenizer=None):
    """
    영어를 한국어로 번역
    
    Args:
        text (str): 번역할 영어 텍스트
        model: 사전 로드된 모델 (없으면 새로 로드)
        tokenizer: 사전 로드된 토크나이저 (없으면 새로 로드)
    
    Returns:
        str: 번역된 한국어 텍스트
    """
    return translate_text(text, 'en', 'ko', model, tokenizer)

In [8]:
if __name__ == "__main__":
    # 모델과 토크나이저 한 번만 로드
    model, tokenizer = load_madlad_model()
    
    # 샘플 텍스트
    korean_text = "인공지능은 인간의 학습능력, 추론능력, 지각능력, 자연언어 이해능력 등을 컴퓨터 프로그램으로 구현한 기술이다."
    english_text = "Artificial intelligence is technology that implements human learning ability, reasoning ability, perception ability, and natural language understanding ability as computer programs."
    
    # 한국어 → 영어 번역
    print(f"[한국어 원문] {korean_text}")
    start_time = time.time()
    english_translation = ko_to_en(korean_text, model, tokenizer)
    ko_to_en_time = time.time() - start_time
    print(f"[영어 번역] {english_translation}")
    print(f"번역 시간: {ko_to_en_time:.2f}초")
    
    # 영어 → 한국어 번역
    print(f"\n[영어 원문] {english_text}")
    start_time = time.time()
    korean_translation = en_to_ko(english_text, model, tokenizer)
    en_to_ko_time = time.time() - start_time
    print(f"[한국어 번역] {korean_translation}")
    print(f"번역 시간: {en_to_ko_time:.2f}초")

[한국어 원문] 인공지능은 인간의 학습능력, 추론능력, 지각능력, 자연언어 이해능력 등을 컴퓨터 프로그램으로 구현한 기술이다.
[영어 번역] Artificial intelligence (AI) is a term used to describe the ability of computer programs to learn, reason, perceive, and understand natural language.
번역 시간: 3.86초

[영어 원문] Artificial intelligence is technology that implements human learning ability, reasoning ability, perception ability, and natural language understanding ability as computer programs.
[한국어 번역] 인공 지능은 인간의 학습 능력, 추론 능력, 지각 능력, 자연 언어 이해 능력을 컴퓨터 프로그램으로 구현하는 기술이다.
번역 시간: 4.06초
